In [ ]:
# Dependencies
import requests
from requests.exceptions import HTTPError
import json
import pandas as pd
import numpy as np
import re
import io
from collections import OrderedDict
from pandas.io.json import json_normalize  
import time 
import urllib.request
from pprint import pprint
import pickle

In [ ]:
## Upload current funder's which is a .txt in json format 

with open("data/prism_current_funder.txt") as datafile:
    funders = []
    for line in datafile:
        funders.append(json.loads(line))

print(type(funders))
print(funders[0])
    

In [ ]:
# current_funder_data = json.loads(funders)
current_funder_df = json_normalize(funders)
current_funder_df.head()

In [ ]:
## Save current funders as an excel file

current_funder_df.to_excel("outputs/current_funder_df.xlsx", index = False, header=True)

In [ ]:
## Upload new funder's which is a .csv 

with open("data/2022_09-09 Northwestern Funders from Funding Institutional.csv", encoding = 'ISO-8859-1') as datafile:
    new_funders_df = pd.read_csv(datafile)
    
new_funders_df.head()

In [ ]:
## Make a list of new funders

new_funder_list = new_funders_df['ï»¿Funder'].tolist()
# print(new_funder_list)

In [ ]:
## Query the ROR API for one result

new_funder = "U.S. Department of Education"

## Other test funders
# "Anesthesia Patient Safety Foundation"
# "Wings for Life"
# "Leona M. and Harry B. Helmsley Charitable Trust"
# "American Brain Tumor Association"

## Optional URLs
## https://api.ror.org/organizations?affiliation={new_funder}
## https://api.ror.org/organizations?query={new_funder}
## https://api.ror.org/organizations?query.advanced=name:{new_funder}
## https://api.ror.org/organizations?query.advanced=aliases:{new_funder}

## Note about pagination
## Affiliation searches return all results, not just the first 20, so results do not need to be paged.
## The ROR API returns a maximum of 20 results per page for query and advanced query searches, beginning at page 1. 
## If metadata.number_of_results is greater than 20, 
## retrieve subsequent records using the format https://api.ror.org/organizations?page=<page number>
## https://ror.readme.io/docs/rest-api#paging

simple_ror_response_list = []


try: 
     
    print("------")
    ror_query_url = f"https://api.ror.org/organizations?affiliation={new_funder}" 
            #print("Requesting", ror_query_url)

    ## Make the API post request 
    ror_response = requests.get(ror_query_url, timeout=30) # params = parameters,
    #print(type(employ_response))
           
    json_ror_data = ror_response.json()
    #print(type(json_ror_data))
    #print(json_ror_data)
    time.sleep(1)
            
    
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  # Python 3.6


except Exception as err:
    print(f'Other error occurred: {err}. ', 'Exiting the loop!')  # Python 3.6
    print(search_response.url)


else:
    print('Success!')

## Append json_employ_data which is a dict to a list, to create a list of dicts. 
simple_ror_response_list.append(json_ror_data.copy())
    

# print(simple_ror_response_list)
        

In [ ]:
## Loop through results to create a smaller dict with needed information

final_funder_list = []

for result_dictionary in simple_ror_response_list:
    
    if not (result_dictionary.get('items') is None):
        substring_dict = result_dictionary.get('items')
#         pprint(substring_dict)
#         pprint(substring_dict[0]['organization'])
        
        for sub_string_items in substring_dict: 
            
            ## Check results
#             print(sub_string_items['substring'])

            ## Create dictionary
            new_funder_dict = {}
            
            ## Add data for search, matching_type, and link
            new_funder_dict["search"] = sub_string_items.get('substring')
            new_funder_dict['matching_type']= sub_string_items.get('matching_type')
            new_funder_dict['link']= sub_string_items['organization'].get('id')
            
            ## Add data for id
            identifier_int = sub_string_items['organization'].get('id')
            identifier_final = identifier_int.replace("https://ror.org/","")
            new_funder_dict['id'] = identifier_final
            
            ## Add data for country
            new_funder_dict['country']= sub_string_items['organization']['country'].get('country_code')
            
            ## Add data in json format for identifiers
            create_id_dict = {}
            create_id_dict['identifier'] = identifier_final
            
            create_ror_dict={}
            create_ror_dict['scheme'] = 'ror'
                        
            create_identifers_list = []
            create_identifers_list.append(create_id_dict)
            create_identifers_list.append(create_ror_dict)
            #print(create_identifers_list)
            new_funder_dict['identifers'] = create_identifers_list
            
            ## Add data for name
            new_funder_dict['name'] = sub_string_items['organization'].get('name')            
            
            ## Add data for title.en
            for item in sub_string_items['organization']['labels']:
                if item['iso639'] == 'en':
                    new_funder_dict['title.en'] =  item['label']
                else:
                    new_funder_dict['title.en'] = sub_string_items['organization'].get('name')   
            
            ## Print the result
            #print(new_funder_dict)
            final_funder_list.append(new_funder_dict)
    
    ## Print the result
    pprint(final_funder_list)
    
    
## Resources
## https://pynative.com/python-check-if-key-exists-in-json-and-iterate-the-json-array/
## https://stackoverflow.com/questions/57211584/how-to-extract-pairs-from-a-dictionary-into-a-new-smaller-one

In [ ]:
## Query the ROR API for "new_funder_list" and get results

## Optional URLs
## https://api.ror.org/organizations?affiliation={new_funder}
## https://api.ror.org/organizations?query={new_funder}
## https://api.ror.org/organizations?query.advanced=name:{new_funder}
## https://api.ror.org/organizations?query.advanced=aliases:{new_funder}

## Note about pagination
## Affiliation searches return all results, not just the first 20, so results do not need to be paged.
## The ROR API returns a maximum of 20 results per page for query and advanced query searches, beginning at page 1. 
## If metadata.number_of_results is greater than 20, 
## retrieve subsequent records using the format https://api.ror.org/organizations?page=<page number>
## https://ror.readme.io/docs/rest-api#paging

ror_response_list = []

for funder in new_funder_list:

    try: 
     
            print("------")
            ror_query_url = f"https://api.ror.org/organizations?affiliation={funder}"  
            print("Requesting", ror_query_url)

            ## Make the API post request 
            ror_response = requests.get(ror_query_url, timeout=30) # params = parameters,
            #print(type(employ_response))
           
            json_ror_data = ror_response.json()
            #print(type(json_ror_data))
            #print(json_ror_data)
            time.sleep(1)
            
    
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6


    except Exception as err:
        print(f'Other error occurred: {err}. ')  # Python 3.6
        print(search_response.url)
 

    else:
        print('Success!')

    ## Append json_ror_data which is a dict to a list, to create a list of dicts. 
    ror_response_list.append(json_ror_data.copy())
    


        

In [ ]:
## Save the results of the ROR API query

with open("outputs/ror_response_list", "wb") as fp:   #Pickling
    pickle.dump(ror_response_list, fp)

with open("outputs/ror_response_list", "rb") as fp:   # Unpickling
    ror_response_list_2 = pickle.load(fp)


In [ ]:
## Check one of the results from the ROR API query

print(ror_response_list_2[280])
#print(len(ror_response_list_2))

In [ ]:
## Loop through results to create a smaller dict with needed information

final_funder_list = []

    
for result_dictionary in ror_response_list_2:

    if not (result_dictionary.get('items') is None):
        substring_dict = result_dictionary.get('items')
#         pprint(substring_dict)
#         pprint(substring_dict[0]['organization'])
       
        for sub_string_items in substring_dict: 

            ## Check results
#             print(sub_string_items['substring'])

            ## Create dictionary
            new_funder_dict = {}

            ## Add data for search, matching_type, and link
            new_funder_dict["search"] = sub_string_items.get('substring')
            new_funder_dict['matching_type']= sub_string_items.get('matching_type')
            new_funder_dict['link']= sub_string_items['organization'].get('id')

            ## Add data for id
            identifier_int = sub_string_items['organization'].get('id')
            identifier_final = identifier_int.replace("https://ror.org/","")
            new_funder_dict['id'] = identifier_final

            ## Add data for country
            new_funder_dict['country']= sub_string_items['organization']['country'].get('country_code')

            ## Add data in json format for identifiers
            create_id_dict = {}
            create_id_dict['identifier'] = identifier_final

            create_ror_dict={}
            create_ror_dict['scheme'] = 'ror'

            create_identifers_list = []
            create_identifers_list.append(create_id_dict)
            create_identifers_list.append(create_ror_dict)
            #print(create_identifers_list)
            new_funder_dict['identifers'] = create_identifers_list

            ## Add data for name
            new_funder_dict['name'] = sub_string_items['organization'].get('name')            

            ## Add data for title.en
            for item in sub_string_items['organization']['labels']:
                if item['iso639'] == 'en':
                    new_funder_dict['title.en'] =  item['label']
                else:
                    new_funder_dict['title.en'] = sub_string_items['organization'].get('name')   
                        
            ## Print the result
#             print(new_funder_dict)
            final_funder_list.append(new_funder_dict)
        
    print(final_funder_list)
## Resources
## https://pynative.com/python-check-if-key-exists-in-json-and-iterate-the-json-array/
## https://stackoverflow.com/questions/57211584/how-to-extract-pairs-from-a-dictionary-into-a-new-smaller-one

In [ ]:
## Save the results of creating smaller dicts

with open("outputs/final_funder_list", "wb") as fp:   #Pickling
    pickle.dump(final_funder_list, fp)

with open("outputs/final_funder_list", "rb") as fp:   # Unpickling
    final_funder_list_2 = pickle.load(fp)


In [ ]:
## Check the result of the smaller dict

print(final_funder_list_2[167])

In [ ]:
## Convert list of dicts to pandas dataframe

final_funder_df = pd.DataFrame(final_funder_list_2)
final_funder_df.head()

In [ ]:
## Export file to excel, with the Pandas index and header

final_funder_df.to_excel("outputs/final_funder_df.xlsx", header=True)